In [1]:
import pandas as pd
from tqdm import tqdm
from itables import show as table_show

In [ ]:
ENTREPOT_PATH = '~/Bureau/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [3]:
#obtention des donnnées de la CAN :
directory = '~/Bureau/Exports/20240524/non_anonyme/'
file = 'AGS_20240524_exports_performance_realise_echelle_parcelle.csv'
df_can = pd.read_csv(directory+file, sep='@')

/tmp/ipykernel_166099/4278151922.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_can = pd.read_csv(directory+file, sep='@')


In [4]:
df_can.columns[0:50]

Index(['performance_id', 'date_calcul', 'nom_domaine_exploitation',
       'id_domaine', 'annee', 'domaine_type', 'departement', 'id_dispositif',
       'type_dispositif', 'nom_reseau_it', 'nom_reseau_ir', 'filiere_sdc',
       'nom_sdc', 'id_sdc', 'num_dephy', 'type_conduite_sdc', 'sdc_valide',
       'parcelle', 'parcelle_id', 'especes', 'varietes', 'rendement',
       'approche_de_calcul', 'zone_id', 'parcelle_edaplos_issuer_id',
       'ift_a_l_ancienne_ift_chimique_total',
       'ift_a_l_ancienne_ift_chimique_tot_hts', 'ift_a_l_ancienne_ift_h',
       'ift_a_l_ancienne_ift_f', 'ift_a_l_ancienne_ift_i',
       'ift_a_l_ancienne_ift_ts', 'ift_a_l_ancienne_ift_a',
       'ift_a_l_ancienne_ift_hh', 'ift_a_l_ancienne_ift_biocontrole',
       'ift_a_l_ancienne_recours_aux_moyens_biologiques',
       'ift_a_l_ancienne_tx_completion',
       'ift_a_l_ancienne_detail_champs_non_renseig',
       'ift_a_la_cible_non_mil_ift_chimique_total',
       'ift_a_la_cible_non_mil_ift_chimique_tot_ht

In [5]:
df_studied = df_can.sample(10)

In [7]:
table_show(pd.melt(df_studied[['especes', 'varietes', 'rendement', 'parcelle_id']], id_vars=['parcelle_id']))

In [6]:
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

# get_intervention_synthetise_outils_can_context --> intervention_synthetise
# get_intervention_synthetise_action_outils_can --> 'intervention_synthetise', 'action_synthetise'
# get_intervention_synthetise_combinaison_outils_can -->  'intervention_synthetise', 'intervention_synthetise_restructure', 'combinaison_outil', 'materiel', 'combinaison_outil_materiel'
# get_intervention_synthetise_culture_outils_can --> 'intervention_synthetise', 'noeuds_synthetise', 'connection_synthetise', 
        # 'plantation_perenne_phases_synthetise', 
        # 'plantation_perenne_synthetise', 'composant_culture_concerne_intervention_synthetise', 
        # 'noeuds_synthetise_restructure', 'plantation_perenne_synthetise_restructure',
        # 'ccc_intervention_synthetise_restructure', 'composant_culture',
        # 'espece', 'variete', 'connection_synthetise_restructure'
# get_intervention_synthetise_culture_prec_outils_can --> 'composant_culture', 'espece', 'variete', 'intervention_synthetise', 
        # 'noeuds_synthetise', 'connection_synthetise', 'culture', 
        # 'noeuds_synthetise_restructure'

tables_entrepot = [
    'zone', 'parcelle', 'sdc', 'domaine', 'culture', 'composant_culture', 'espece', 'variete', 'noeuds_realise', 
    'plantation_perenne_phases_realise', 'plantation_perenne_realise', 'recolte_rendement_prix', 'action_realise', 
    'action_realise_agrege', 'composant_culture_concerne_intervention_realise', 'connection_realise','recolte_rendement_prix_restructure'
]

tables_performance = [
    'parcelle_realise_performance'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id',verbose=False) 
import_dfs(tables_performance, ENTREPOT_PATH, sep = ',',verbose=False) 

100%|██████████| 1/1 [00:05<00:00,  5.62s/it]


In [6]:

import_dfs(['recolte_rendement_prix_restructure'], ENTREPOT_PATH, sep = ',',index_col='id',verbose=False) 


100%|██████████| 1/1 [00:00<00:00,  1.34it/s]


In [7]:
studied_ids = [
    'fr.inra.agrosyst.api.entities.Plot_767b5d1d-3d66-4383-8f58-8863dcdc0c5a',
    'fr.inra.agrosyst.api.entities.Plot_b2a65257-aa63-4c05-91b9-558919b4ae0b',
    'fr.inra.agrosyst.api.entities.Plot_d0583d18-5a2c-4fa1-86e6-09d9071974b9'
]

In [8]:
df['parcelle_test'] = df['parcelle'].loc[
    df['parcelle'].index.isin(studied_ids)
]
df['zone_test'] = df['zone'].loc[
    df['zone']['parcelle_id'].isin(df['parcelle_test'].index)
]
df['noeuds_realise_test'] = df['noeuds_realise'].loc[
    df['noeuds_realise']['zone_id'].isin(df['zone_test'].index)
]
df['connection_realise_test'] = df['connection_realise'].loc[
    df['connection_realise']['cible_noeuds_realise_id'].isin(df['noeuds_realise_test'].index)
]
df['plantation_perenne_realise_test'] = df['plantation_perenne_realise'].loc[
    df['plantation_perenne_realise']['zone_id'].isin(df['zone_test'].index)
]
df['plantation_perenne_phases_realise_test'] = df['plantation_perenne_phases_realise'].loc[
    df['plantation_perenne_phases_realise']['plantation_perenne_realise_id'].isin(df['plantation_perenne_realise_test'].index)
]
df['culture_test'] = df['culture'].loc[
    (df['culture'].index.isin(df['noeuds_realise_test']['culture_id'])) |
    (df['culture'].index.isin(df['plantation_perenne_realise_test']['culture_id']))  |
    (df['culture'].index.isin(df['connection_realise_test']['culture_intermediaire_id']))
]
df['composant_culture_test'] = df['composant_culture'].loc[
    (df['composant_culture']['culture_id'].isin(df['culture_test'].index))
]
df['composant_culture_concerne_intervention_realise_test'] = df['composant_culture_concerne_intervention_realise'].loc[
    (df['composant_culture_concerne_intervention_realise']['composant_culture_id'].isin(df['composant_culture_test'].index))
]
df['espece_test'] = df['espece'].loc[
    df['espece'].index.isin(df['composant_culture_test']['espece_id'])
]
df['variete_test'] = df['variete'].loc[
    df['variete'].index.isin(df['composant_culture_test']['variete_id'])
]
df['action_realise_agrege_test'] = df['action_realise_agrege'].loc[
    df['action_realise_agrege']['parcelle_id'].isin(studied_ids)
]

df['recolte_rendement_prix_test'] = df['recolte_rendement_prix'].loc[
    df['recolte_rendement_prix']['action_id'].isin(df['action_realise_agrege_test'].index)
]

df['recolte_rendement_prix_restructure_test'] = df['recolte_rendement_prix_restructure'].loc[
    df['recolte_rendement_prix_restructure'].index.isin(df['recolte_rendement_prix_test'].index)
]

df['action_realise_test'] = df['action_realise'].loc[
    df['action_realise'].index.isin(df['action_realise_agrege_test'].index)
]

In [10]:
path='./2/'
df['zone_test'].to_csv(path+'zone.csv')
df['composant_culture_test'].to_csv(path+'composant_culture.csv')
df['noeuds_realise_test'].to_csv(path+'noeuds_realise'+'.csv')
df['connection_realise_test'].to_csv(path+'connection_realise'+'.csv')
df['espece_test'].to_csv(path+'espece'+'.csv')
df['variete_test'].to_csv(path+'variete'+'.csv')
df['culture_test'].to_csv(path+'culture'+'.csv')
df['plantation_perenne_realise_test'].to_csv(path+'plantation_perenne_realise'+'.csv')
df['plantation_perenne_phases_realise_test'].to_csv(path+'plantation_perenne_phases_realise'+'.csv')
df['parcelle_test'].to_csv(path+'parcelle'+'.csv')
df['action_realise_agrege_test'].to_csv(path+'action_realise_agrege.csv')
df['recolte_rendement_prix_test'].to_csv(path+'recolte_rendement_prix.csv')
df['action_realise_test'].to_csv(path+'action_realise.csv')
df['recolte_rendement_prix_test'].to_csv(path+'recolte_rendement_prix.csv')
df['composant_culture_concerne_intervention_realise_test'].to_csv(path+'composant_culture_concerne_intervention_realise.csv')
df['recolte_rendement_prix_restructure_test'].to_csv(path+'recolte_rendement_prix_restructure.csv')